# Transfer Learning

En pratique, très peu de personnes entraînent des CNNs complets à partir de zéro (avec une initialisation aléatoire), car il est relativement rare de disposer d'un ensemble de données de taille suffisante. 
Au lieu de cela, il est courant d'utiliser des CNNs pré-entrainés sur un ensemble de données très volumineux (par exemple, ImageNet, qui contient 1,2 million d'images réparties en 1000 catégories), puis d'utiliser le CNN soit comme une initialisation, soit comme un extracteur de caractéristiques fixe pour la tâche d'intérêt. Ainsi, il y a deux moyens d'utiliser le transfert learning : 

 - CNN en tant qu'extracteur de caractéristiques fixes : Prenez un CNN pré-entraîné, retirez la dernière couche fully conneccted, puis traitez le reste du CNN comme un extracteur de caractéristiques fixes pour le nouveau jeu de données. Vous ajouter ensuite votre propre couche fully connected et vous entrainez seulement ça.

 - Finetuner le CNN : La deuxième stratégie consiste non seulement à remplacer et ré-entraîner le classificateur sur le dessus du CNN avec le nouveau jeu de données, mais aussi à entrainer tout le réseau. Il est possible de "finetuner" toutes les couches du ConvNet, ou de conserver certaines des couches plus anciennes fixes.

Dans ce TP on va utiliser la première methode, mais la deuxième s'implémente de manière similaire

# Données

In [52]:
import numpy as np
from tqdm import tqdm
import cv2
import os
from torchvision import datasets, models, transforms
import torch
import torchvision
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.optim as optim


# Données

Nous allons classifier un modèle pour reconnaitre si l'image d'entrée est celle d'une abeille ou d'une fourmis.

Pour commencer telecharger ce dataset (https://download.pytorch.org/tutorial/hymenoptera_data.zip) et dezipez le sur le dossier

En executant le code ci-dessus vous allez obtenir des dataloaders pour votre ensemble de train et de test. La particularité ici c'est que nous avons très peu des données. A priori, il serait très difficile d'éntrainer un bon modèle avec si peu de données mais grâce au Transfer Learning, c'est possible d'obtenir une bonne précision.

In [69]:
data_dir = 'hymenoptera_data'


data_transforms_train = transforms.Compose([
        transforms.CenterCrop(224),
        transforms.ToTensor()])

data_transforms_train = transforms.Compose([
        transforms.CenterCrop(224),
        transforms.ToTensor()])

image_datasets_train =  datasets.ImageFolder(os.path.join(data_dir, 'train'), data_transforms_train)
image_datasets_val =  datasets.ImageFolder(os.path.join(data_dir, 'val'), data_transforms_val)

dataloaders_train = torch.utils.data.DataLoader(image_datasets_train, batch_size=4, shuffle=True)
dataloaders_val = torch.utils.data.DataLoader(image_datasets_val, batch_size=4, shuffle=True)


class_names = image_datasets_train.classes

## Exercice 1

Visualiser quelques images du dataset et normalisez les images

# Utilisation d'un autre modèle

Ici, nous allons utiliser resnet-18 comme notre modème de base.  Nous allons freeze tout ses layers et on va ajouter une couche linéaire supplémentaire à la fin, que nous allons entrainé. Ainsi, resnet-18 sert comme du feature-extraction qu'on apprend ensuite à classifier comme abeille ou fourmis.

In [48]:
# Chargeons resnet18
model_conv = torchvision.models.resnet18(weights='IMAGENET1K_V1')
model_conv

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

Vous pouvez visualiser maintenant toute l'architecture du CNN. Nous allons freeze tous les paramètres du réseaux, pour ne pas entrainer, et nous allons changer la sortie dernière couche (la fc) de 1000 out_features à juste 2 (pour nos 2 classes)

In [49]:

# Freeze toutes les couches
for param in model_conv.parameters():
    param.requires_grad = False

# Ajoute une couche de classification à la fin (qui sera entrainée)
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2) # on change la dernière couche de classification pour avoir 2 classes



Regardons maintenant notre model_conv. La dernière couche a bien été changé ! 

In [51]:
model_conv

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

Note : Actuellement le modèle est en CPU. Vous pouvez tout passer en GPU pour faire un entraînement plus rapide

Cependant même sur CPU l'entraînement n'est pas très long parce que on n'a très peu des paramètres à changer.

# Entraînement

## Exercice 2
Implémenter une boucle d'entraînement classique pour notre petit problème. Pensez à calculer la val_loss à chaque epoch (ou l'accuracy si vous voulez)


In [ ]:
## Your code goes here ##

# Visualisation

Vous pouvez utiliser cette fonction pour visualiser les prédictions de votre modèle


In [ ]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            # inputs = inputs.to(device) si GPU
            # labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title(f'predicted: {class_names[preds[j]]}')
                plt.imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)